In [ ]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [ ]:
!python -m spacy download en_core_web_lg

2022-10-02 15:02:28.689691: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import gensim

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
df = pd.read_csv('/content/spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [ ]:
df.rename({'v1': 'category', 'v2': 'mail'}, axis=1, inplace=True)

In [ ]:
df.head()

,category,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.mail[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
df.category.value_counts()

ham     4825
spam     747
Name: category, dtype: int64

In [ ]:
def spacy_process(text):
    filtered = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered.append(token.lemma_)
    return filtered

In [ ]:
mail_processed = df['mail'].apply(spacy_process)

In [ ]:
mail_processed[0]

['jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'Cine',
 'get',
 'amore',
 'wat']

In [ ]:
cbow_model = gensim.models.Word2Vec(size=300, min_count=2)
sgram_model = gensim.models.Word2Vec(size=300, min_count=2, sg=1)

In [ ]:
cbow_model.build_vocab(mail_processed)

In [ ]:
sgram_model.build_vocab(mail_processed)

In [ ]:
cbow_model.corpus_count

5572

In [ ]:
sgram_model.corpus_count

5572

In [ ]:
cbow_model.epochs

5

In [ ]:
sgram_model.epochs

5

In [ ]:
cbow_model.train(mail_processed, total_examples=cbow_model.corpus_count, 
                 epochs=cbow_model.epochs)

(191896, 238315)

In [ ]:
sgram_model.train(mail_processed, total_examples=sgram_model.corpus_count, 
                  epochs=sgram_model.epochs)

(191810, 238315)

In [ ]:
print('cbow model vocabulary size:', len(cbow_model.wv.vocab))

print('skgram model vocabulary size:', len(sgram_model.wv.vocab))

cbow model vocabulary size: 4081
skgram model vocabulary size: 4081


In [ ]:
cbow_model.wv.most_similar('great')

[('good', 0.9999580979347229),
 ('text', 0.9999566078186035),
 ('d', 0.9999562501907349),
 ('love', 0.9999552965164185),
 ('like', 0.9999548196792603),
 ('go', 0.9999541640281677),
 ('want', 0.99995356798172),
 ('ur', 0.9999529123306274),
 ('day', 0.9999528527259827),
 ('need', 0.9999527335166931)]

In [ ]:
sgram_model.wv.most_similar('great')

[('morning', 0.9994887709617615),
 ('s', 0.9992040395736694),
 ('dear', 0.9989128112792969),
 ('tomorrow', 0.9988532662391663),
 ('m', 0.9988387227058411),
 ('way', 0.9987601041793823),
 ('day', 0.9987301230430603),
 ('will', 0.9986969232559204),
 ('oh', 0.9986810684204102),
 ('to', 0.9986562132835388)]

In [ ]:
for token in mail_processed[1]:
    if token in sgram_model.wv.vocab:
        print(token)

ok
lar
joke
wif
u
oni


In [ ]:
(np.mean(sgram_model.wv.word_vec('go'), axis=0)).tolist()

0.0066011506132781506

In [ ]:
file_name = '/content/drive/MyDrive/Ml_course/NLP_sessions/GoogleNews-vectors-negative300.bin'

In [ ]:
google_model = gensim.models.KeyedVectors.load_word2vec_format(file_name, binary=True)

KeyboardInterrupt: ignored

In [ ]:
def get_embedding_w2v(doc_tokens, pre_trained=0, cbow=0):
    '''doc_tokens: list of words,
    pre_trained: int 0 or 1; if 1 use google model
    cbow: int 0 or 1: if 1 use cbow; default is skipgram model'''
    embeddings = []
    if cbow:
        pre_trained = 0
    if pre_trained:
        model = google_model
    elif cbow:
        model = cbow_model
    else:
        model = sgram_model
    # iterate over tokens to extract their vectors    
    for tok in doc_tokens:
        if tok in model.wv.vocab:
            embeddings.append(model.wv.word_vec(tok))
    # mean the vectors of individual words to get the vector of the statement
    return np.mean(embeddings, axis=0)

In [ ]:
google_vectors = mail_processed.apply(lambda x: get_embedding_w2v(x))
google_vectors = pd.DataFrame(google_vectors.tolist())
print('X shape:', google_vectors.shape)

TypeError: ignored

In [ ]:
google_vectors

0       [-0.023970269, -0.08539971, 0.026500802, 0.046...
1       [-0.07740392, -0.0884452, 0.047572274, 0.05087...
2       [0.07374151, -0.08042842, 0.09463927, -0.06241...
3       [-0.09185543, -0.11664696, 0.05684878, 0.09473...
4       [-0.015822953, -0.110719144, 0.024480356, 0.08...
                              ...                        
5567    [0.07114995, -0.064738415, 0.0080614565, 0.039...
5568    [-0.039234262, -0.08500642, 0.055060048, 0.030...
5569    [-0.0022907848, -0.11255865, 0.0028063362, 0.0...
5570    [0.003552802, -0.1052001, 0.033180673, 0.04258...
5571    [0.002378621, -0.09587888, 0.009989627, 0.0359...
Name: mail, Length: 5572, dtype: object